In [68]:
%matplotlib inline
# A linha acima é necessária para visualizar os gráficos no jupyter notebook

import os
import pandas as pd
# Módulo com funções matemáticas para arrays multidimensionais (matrizes)
import numpy as np
# Módulo com funções de geração de gráficos
import matplotlib.pyplot as plt


In [92]:
# Carregando os dados
target_carne_produzida = pd.read_csv('meat-production-tonnes.csv')
pais_carne_produzida = target_carne_produzida['Entity']

target_carne_produzida.head(5)


,Entity,Code,Year,"Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)"
0,Afghanistan,AFG,1961,129420
1,Afghanistan,AFG,1962,132206
2,Afghanistan,AFG,1963,138971
3,Afghanistan,AFG,1964,143830
4,Afghanistan,AFG,1965,150195


In [93]:
producao_2010 = target_carne_produzida['Year'] == 2010
target_carne_produzida.loc[producao_2010, :]

,Entity,Code,Year,"Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)"
49,Afghanistan,AFG,2010,328160
107,Africa,NaN,2010,16638514
165,Albania,ALB,2010,91440
223,Algeria,DZA,2010,635153
281,American Samoa,ASM,2010,340
...,...,...,...,...
13289,Western Sahara,ESH,2010,2242
13347,World,OWID_WRL,2010,294619190
13405,Yemen,YEM,2010,323496
13494,Zambia,ZMB,2010,265615


In [94]:
feature_consumo_per_capita = pd.read_csv('per-capita-meat-consumption-by-type-kilograms-per-year.csv')

producao_2010 = feature_consumo_per_capita['Year'] == 2010
consumo_2010_per_cpt = feature_consumo_per_capita.loc[producao_2010, :]
consumo_2010_per_cpt = consumo_2010_per_cpt.loc[:, ['Entity', 'Year', 'Beef and bufallo (kilograms per year)']]
consumo_2010_per_cpt

,Entity,Year,Beef and bufallo (kilograms per year)
49,Afghanistan,2010,4.70
106,Africa,2010,6.31
159,Albania,2010,21.38
216,Algeria,2010,4.60
273,Americas,2010,29.74
...,...,...,...
10769,Western Asia,2010,8.16
10822,World,2010,9.53
10875,Yemen,2010,4.96
10963,Zambia,2010,5.64


In [95]:
feature_IDH = pd.read_csv('HDI.csv')
IDH_dados_importantes = feature_IDH.loc[:,['Id', 'Country', 'HDI', 'Life expectancy', 'Mean years of schooling', 'Coefficient of human inequality']]

IDH_dados_importantes

,Id,Country,HDI,Life expectancy,Mean years of schooling,Coefficient of human inequality
0,1,Norway,0.949,81.7,12.7,5.4
1,2,Australia,0.939,82.5,13.2,8.0
2,3,Switzerland,0.939,83.1,13.4,8.4
3,4,Germany,0.926,81.1,13.2,7.0
4,5,Denmark,0.925,80.4,12.7,7.0
...,...,...,...,...,...,...
190,191,Monaco,NaN,NaN,NaN,NaN
191,192,Nauru,NaN,NaN,NaN,NaN
192,193,San Marino,NaN,NaN,NaN,NaN
193,194,Somalia,NaN,55.7,NaN,NaN


In [96]:
feature_desemprego_pais = pd.read_csv('API_ILO_country_YU.csv')

desemprego_2010 = feature_desemprego_pais.loc[:,['Country Name', '2010']]
desemprego_2010


,Country Name,2010
0,Afghanistan,20.600000
1,Angola,10.800000
2,Albania,25.799999
3,Arab World,25.022214
4,United Arab Emirates,9.800000
...,...,...
214,"Yemen, Rep.",30.299999
215,South Africa,50.799999
216,"Congo, Dem. Rep.",12.300000
217,Zambia,26.400000


In [97]:
feature_pib = pd.read_csv('GDP.csv')

pib_2010 = feature_pib.loc[:,['Country ', '2010']]
pib_2010

,Country,2010
0,Aruba,33732.847450
1,Afghanistan,1637.377987
2,Angola,6230.297028
3,Albania,9628.025783
4,Arab World,14127.778020
...,...,...
255,Kosovo,7782.875107
256,"Yemen, Rep.",4472.726448
257,South Africa,11728.160490
258,Zambia,3269.794329


In [98]:
feature_dados_commodity = pd.read_csv('commodity_trade_statistics_data.csv') 
ano_2010 = feature_dados_commodity['year'] == 2010
commodity_2010 = feature_dados_commodity.loc[ano_2010, :]
commodity_2010 = commodity_2010.loc[:, ['country_or_area', 'year', 'commodity', 'flow', 'trade_usd', 'weight_kg', 'category']]
commodity_2010

C:\Users\T-Gamer\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country_or_area,year,commodity,flow,trade_usd,weight_kg,category
69,Albania,2010,"Horses, live except pure-bred breeding",Import,3579,1119.0,01_live_animals
70,Albania,2010,"Bovine animals, live pure-bred breeding",Import,628861,197482.0,01_live_animals
71,Albania,2010,"Bovine animals, live, except pure-bred breeding",Import,17658038,8538249.0,01_live_animals
72,Albania,2010,"Swine, live pure-bred breeding",Import,90783,55980.0,01_live_animals
73,Albania,2010,"Swine, live except pure-bred breeding < 50 kg",Import,199919,123788.0,01_live_animals
...,...,...,...,...,...,...,...
8225770,Yemen,2010,ALL COMMODITIES,Re-Export,237950133,NaN,all_commodities
8225799,Zambia,2010,ALL COMMODITIES,Import,5320834326,NaN,all_commodities
8225800,Zambia,2010,ALL COMMODITIES,Export,7200267057,NaN,all_commodities
8225844,Zimbabwe,2010,ALL COMMODITIES,Import,5852266737,NaN,all_commodities
